In [1]:
import numpy as np
from ortools.sat.python import cp_model

In [2]:
# Initial sudoku grid
initial_grid = [
    [0, 6, 0, 0, 5, 0, 0, 2, 0],
    [0, 0, 0, 3, 0, 0, 0, 9, 0],
    [7, 0, 0, 6, 0, 0, 0, 1, 0],
    [0, 0, 6, 0, 3, 0, 4, 0, 0],
    [0, 0, 4, 0, 7, 0, 1, 0, 0],
    [0, 0, 5, 0, 9, 0, 8, 0, 0],
    [0, 4, 0, 0, 0, 1, 0, 0, 6],
    [0, 3, 0, 0, 0, 8, 0, 0, 0],
    [0, 2, 0, 0, 4, 0, 0, 5, 0],
]

# Size of the main grid
N = 9
# Size of subgrid
S = 3

In [3]:
# Initialize the CP-SAT instances
solver = cp_model.CpSolver()
model = cp_model.CpModel()

# Create the variables
variables = {}
for row in range(len(initial_grid)):
    for col in range(len(initial_grid[row])):
        # Create a new variable that may take any integer value between 1 and 9
        variables[row, col] = model.new_int_var(1, 9, f"{(row, col)}")


# Add the model's constraints

# All different on rows
for row in range(N):
    row_elements = [v for k, v in variables.items() if k[0] == row]
    model.add_all_different(row_elements)

# All different on cols
for col in range(N):
    col_elements = [v for k, v in variables.items() if k[1] == col]
    model.add_all_different(col_elements)

# All different in the subgrids
for i in range(S):
    for j in range(S):
        start_row = i * S
        start_col = j * S
        cell_elements = [
            variables[r, c]
            for r in range(start_row, start_row + S)
            for c in range(start_col, start_col + S)
        ]
        model.add_all_different(cell_elements)

# Initial values
for row in range(N):
    for col in range(N):
        el = initial_grid[row][col]
        if el != 0:
            model.add(variables[row, col] == el)

# Solve the model
solver.solve(model)
solver.status_name()

'OPTIMAL'

In [4]:
# Retreive the solution
solution = [row.copy() for row in initial_grid]
for row in range(N):
    for col in range(N):
        solution[row][col] = solver.value(variables[row, col])

solution

[[8, 6, 1, 4, 5, 9, 7, 2, 3],
 [4, 5, 2, 3, 1, 7, 6, 9, 8],
 [7, 9, 3, 6, 8, 2, 5, 1, 4],
 [2, 1, 6, 8, 3, 5, 4, 7, 9],
 [9, 8, 4, 2, 7, 6, 1, 3, 5],
 [3, 7, 5, 1, 9, 4, 8, 6, 2],
 [5, 4, 7, 9, 2, 1, 3, 8, 6],
 [1, 3, 9, 5, 6, 8, 2, 4, 7],
 [6, 2, 8, 7, 4, 3, 9, 5, 1]]